Informative text

In [27]:
import psycopg2
import os
import pandas as pd

# Establish database connection
conn = psycopg2.connect(
    dbname=os.environ.get('DBNAME'), 
    user=os.environ.get('DBUSER'), 
    password=os.environ.get('DBPASS'), 
    host=os.environ.get('DBHOST'), 
    port=os.environ.get('DBPORT')
)

# Function to query a table and display the first few rows
def display_table(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, conn)
    print(f"First few rows of table '{table_name}':")
    print(df.head(), "\n")  # Display the first few rows

# Display the first few rows of each table
display_table("targetranges")
display_table("depositrates")
display_table("targetrates")

# Close the database connection
conn.close()


First few rows of table 'targetranges':
   id       date  lower_bound  upper_bound
0   1 2000-01-03         1.25         2.25
1   2 2000-01-04         1.25         2.25
2   3 2000-01-05         1.25         2.25
3   4 2000-01-06         1.25         2.25
4   5 2000-01-07         1.25         2.25 

First few rows of table 'depositrates':
   id       date  value
0   1 1999-12-01   1.19
1   2 2000-01-01   1.19
2   3 2000-02-01   1.27
3   4 2000-03-01   1.35
4   5 2000-04-01   1.35 

First few rows of table 'targetrates':
   id       date  value
0   1 2019-06-01  -0.75
1   2 2019-07-01  -0.75
2   3 2019-08-01  -0.75
3   4 2019-09-01  -0.75
4   5 2019-10-01  -0.75 



/tmp/ipykernel_5084/2949156054.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_5084/2949156054.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_5084/2949156054.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Create data frame with midpoint of the range for 2000-2019

In [28]:
import os
import pandas as pd
from sqlalchemy import create_engine

# Set up the database connection using SQLAlchemy
db_string = f"postgresql://{os.environ.get('DBUSER')}:" \
            f"{os.environ.get('DBPASS')}@{os.environ.get('DBHOST')}:" \
            f"{os.environ.get('DBPORT')}/{os.environ.get('DBNAME')}"
engine = create_engine(db_string)

# SQL query to calculate the average midpoint for each month, shifted to the first of the next month,
# plus the special case for January 1st, 2000
query = '''
WITH MonthlyAverages AS (
  SELECT
    DATE_TRUNC('month', date) + INTERVAL '1 month' - INTERVAL '1 day' AS date,
    AVG((lower_bound + upper_bound) / 2.0) AS value
  FROM
    targetranges
  GROUP BY
    DATE_TRUNC('month', date)
)
SELECT
  (date + INTERVAL '1 day')::date AS date,
  value
FROM
  MonthlyAverages

UNION ALL

SELECT
  '2000-01-01'::date AS report_date,
  (SELECT (lower_bound + upper_bound) / 2.0
   FROM targetranges
   ORDER BY date ASC
   LIMIT 1) AS value
ORDER BY
  date;
'''

# Read the data into a Pandas DataFrame
SNB_old = pd.read_sql_query(query, engine)

# Close the connection
engine.dispose()

# Show the resulting DataFrame
print(SNB_old)


           date     value
0    2000-01-01  1.750000
1    2000-02-01  1.750000
2    2000-03-01  2.202381
3    2000-04-01  2.478261
4    2000-05-01  3.000000
..          ...       ...
230  2019-03-01 -0.750000
231  2019-04-01 -0.750000
232  2019-05-01 -0.750000
233  2019-06-01 -0.750000
234  2019-07-01 -0.750000

[235 rows x 2 columns]


Create dataframe SNB_new which pulls the target from 2019-2023, and shifts forward by one month to reflect the average from the previous month. Delete the first entry (June, reported one July 1st)

In [32]:
import os
import pandas as pd
from sqlalchemy import create_engine

# Set up the database connection using SQLAlchemy
db_string = f"postgresql://{os.environ.get('DBUSER')}:" \
            f"{os.environ.get('DBPASS')}@{os.environ.get('DBHOST')}:" \
            f"{os.environ.get('DBPORT')}/{os.environ.get('DBNAME')}"
engine = create_engine(db_string)

# SQL query to select date and value from targetrates
query = "SELECT date, value FROM targetrates;"

# Read the data into a Pandas DataFrame called SNB_new
SNB_new = pd.read_sql_query(query, engine)

# Close the connection
engine.dispose()

# Convert the 'date' column to datetime if it's not already
SNB_new['date'] = pd.to_datetime(SNB_new['date'])

# Shift the 'date' forward by one month
SNB_new['date'] = SNB_new['date'] + pd.DateOffset(months=1)

# Drop the first row
SNB_new = SNB_new.iloc[1:].reset_index(drop=True)

# SNB_new now contains the date shifted by one month and without the first row
print(SNB_new)  # To verify the output


           date  value
0    2019-08-01  -0.75
1    2019-09-01  -0.75
2    2019-10-01  -0.75
3    2019-11-01  -0.75
4    2019-12-01  -0.75
...         ...    ...
5119 2019-07-06  -0.75
5120 2019-07-07  -0.75
5121 2019-07-10  -0.75
5122 2019-07-11  -0.75
5123 2019-07-12  -0.75

[5124 rows x 2 columns]
